### 1. Import libraries

In [10]:
import cv2 
import numpy as np 
import tensorflow.keras.backend as K
import tensorflow as tf
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.models import Model
from tensorflow.keras.layers import *
from tensorflow.keras.datasets import cifar10 
from tensorflow.keras.utils import to_categorical 
from tensorflow.keras.optimizers import SGD 
from tensorflow.keras.callbacks import LearningRateScheduler

In [11]:
num_classes = 10

def load_cifar10_data(img_rows, img_cols):

   (X_train, Y_train), (X_valid, Y_valid) = cifar10.load_data()

   X_train = np.array([cv2.resize(img, (img_rows,img_cols)) for img in X_train])
   X_valid = np.array([cv2.resize(img, (img_rows,img_cols)) for img in X_valid[:,:,:,:]])

   Y_train = to_categorical(Y_train, num_classes)
   Y_valid = to_categorical(Y_valid, num_classes)
   X_train = X_train.astype('float32')
   X_valid = X_valid.astype('float32')

    # preprocess data
   X_train = X_train / 255.0
   X_valid = X_valid / 255.0

   return X_train, Y_train, X_valid, Y_valid

In [12]:
X_train, y_train, X_test, y_test = load_cifar10_data(32, 32)

In [13]:
def inception_module(x,
                     filters_1x1,
                     filters_3x3_reduce,
                     filters_3x3,
                     filters_5x5_reduce,
                     filters_5x5,
                     filters_pool_proj,
                     name=None):
    
    conv_1x1 = Conv2D(filters_1x1, (1, 1), padding='same', activation='relu', kernel_initializer=kernel_init, bias_initializer=bias_init)(x)
    
    conv_3x3 = Conv2D(filters_3x3_reduce, (1, 1), padding='same', activation='relu', kernel_initializer=kernel_init, bias_initializer=bias_init)(x)
    conv_3x3 = Conv2D(filters_3x3, (3, 3), padding='same', activation='relu', kernel_initializer=kernel_init, bias_initializer=bias_init)(conv_3x3)

    conv_5x5 = Conv2D(filters_5x5_reduce, (1, 1), padding='same', activation='relu', kernel_initializer=kernel_init, bias_initializer=bias_init)(x)
    conv_5x5 = Conv2D(filters_5x5, (5, 5), padding='same', activation='relu', kernel_initializer=kernel_init, bias_initializer=bias_init)(conv_5x5)

    pool_proj = MaxPool2D((3, 3), strides=(1, 1), padding='same')(x)
    pool_proj = Conv2D(filters_pool_proj, (1, 1), padding='same', activation='relu', kernel_initializer=kernel_init, bias_initializer=bias_init)(pool_proj)

    output = concatenate([conv_1x1, conv_3x3, conv_5x5, pool_proj], axis=3, name=name)
    
    return output

In [14]:
kernel_init = tf.keras.initializers.glorot_uniform()
bias_init = tf.keras.initializers.Constant(value=0.2)

In [15]:
input_layer = Input(shape=(32, 32, 3))

x = Conv2D(64, (7, 7), padding='same', strides=(2, 2), activation='relu', name='conv_1_7x7/2', kernel_initializer=kernel_init, bias_initializer=bias_init)(input_layer)
x = MaxPool2D((3, 3), padding='same', strides=(2, 2), name='max_pool_1_3x3/2')(x)
#x = Conv2D(64, (1, 1), padding='same', strides=(1, 1), activation='relu', name='conv_2a_3x3/1')(x)
x = Conv2D(192, (3, 3), padding='same', strides=(1, 1), activation='relu', name='conv_2b_3x3/1')(x)
x = MaxPool2D((3, 3), padding='same', strides=(2, 2), name='max_pool_2_3x3/2')(x)

x = inception_module(x,
                     filters_1x1=64,
                     filters_3x3_reduce=96,
                     filters_3x3=128,
                     filters_5x5_reduce=16,
                     filters_5x5=32,
                     filters_pool_proj=32,
                     name='inception_3a')

x = inception_module(x,
                     filters_1x1=128,
                     filters_3x3_reduce=128,
                     filters_3x3=192,
                     filters_5x5_reduce=32,
                     filters_5x5=96,
                     filters_pool_proj=64,
                     name='inception_3b')

x = MaxPool2D((3, 3), padding='same', strides=(2, 2), name='max_pool_3_3x3/2')(x)

x = inception_module(x,
                     filters_1x1=192,
                     filters_3x3_reduce=96,
                     filters_3x3=208,
                     filters_5x5_reduce=16,
                     filters_5x5=48,
                     filters_pool_proj=64,
                     name='inception_4a')


classifier_1 = AveragePooling2D((2, 2), strides=2)(x)
classifier_1 = Conv2D(128, (1, 1), padding='same', activation='relu')(classifier_1)
classifier_1 = Flatten()(classifier_1)
classifier_1 = Dense(128, activation='relu')(classifier_1)
classifier_1 = Dropout(0.5)(classifier_1)
classifier_1 = Dense(10, activation='softmax', name='auxilliary_output_1')(classifier_1)


In [16]:
model = Model(input_layer, classifier_1, name='googlenet')

In [17]:
model.summary()

Model: "googlenet"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 32, 32, 3)]  0                                            
__________________________________________________________________________________________________
conv_1_7x7/2 (Conv2D)           (None, 16, 16, 64)   9472        input_2[0][0]                    
__________________________________________________________________________________________________
max_pool_1_3x3/2 (MaxPooling2D) (None, 8, 8, 64)     0           conv_1_7x7/2[0][0]               
__________________________________________________________________________________________________
conv_2b_3x3/1 (Conv2D)          (None, 8, 8, 192)    110784      max_pool_1_3x3/2[0][0]           
__________________________________________________________________________________________

In [18]:

epochs = 25
initial_lrate = 0.01

def decay(epoch, steps=100):
    initial_lrate = 0.01
    drop = 0.96
    epochs_drop = 8
    lrate = initial_lrate * tf.math.pow(drop, tf.math.floor((1+epoch)/epochs_drop))
    return lrate

sgd = SGD(lr=initial_lrate, momentum=0.9, nesterov=False)

lr_sc = LearningRateScheduler(decay, verbose=1)

model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

history = model.fit(X_train, y_train, validation_data=(X_test, y_test), 
                    epochs=epochs, batch_size=128, callbacks=[lr_sc])


Epoch 1/25

Epoch 00001: LearningRateScheduler reducing learning rate to tf.Tensor(0.01, shape=(), dtype=float32).
391/391 [==============================] - 53s 134ms/step - loss: 2.2419 - accuracy: 0.1354 - val_loss: 1.9231 - val_accuracy: 0.2530
Epoch 2/25

Epoch 00002: LearningRateScheduler reducing learning rate to tf.Tensor(0.01, shape=(), dtype=float32).
391/391 [==============================] - 59s 150ms/step - loss: 1.8345 - accuracy: 0.2888 - val_loss: 1.5535 - val_accuracy: 0.4087
Epoch 3/25

Epoch 00003: LearningRateScheduler reducing learning rate to tf.Tensor(0.01, shape=(), dtype=float32).
391/391 [==============================] - 82s 210ms/step - loss: 1.5993 - accuracy: 0.4003 - val_loss: 1.3759 - val_accuracy: 0.4918
Epoch 4/25

Epoch 00004: LearningRateScheduler reducing learning rate to tf.Tensor(0.01, shape=(), dtype=float32).
391/391 [==============================] - 62s 158ms/step - loss: 1.4413 - accuracy: 0.4751 - val_loss: 1.2938 - val_accuracy: 0.5296
Epoc